# 🏥 SJGHC Hospital Episode Analysis – Reproducible Workflow

## 1. Introduction
This notebook analyzes hospital episode data from St John of God Health Care (SJGHC).  
The goal is to uncover operational, financial, and referral-based insights that support  
better patient care, resource allocation, and cost containment strategies.

In [ ]:
# 📦 Functions

import os
import pandas as pd

def load_or_create_df(version="raw", excel_path=None, sheet_name=0):
    """
    Loads a versioned pickle file if it exists.
    If not, loads from Excel and saves the pickle for future use.
    
    Parameters:
    - version: str, one of ["raw", "cleansed", "transformed"]
    - excel_path: str, path to Excel file (required if pickle doesn't exist)
    - sheet_name: str or int, sheet name or index in Excel file
    
    Returns:
    - df: pandas DataFrame
    """
    pickle_path = f"../data/staging/hcp_{version}.pkl"
    
    if os.path.exists(pickle_path):
        print(f"✅ Loading '{version}' version from pickle...")
        df = pd.read_pickle(pickle_path)
    else:
        if not excel_path:
            raise ValueError("Excel path must be provided if pickle doesn't exist.")
        print(f"⚠️ Pickle not found. Loading from Excel and saving as '{version}'...")
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        df.to_pickle(pickle_path)
    
    return df

## 2. Data Import

In [4]:
# 🚀 Main Workflow

# Load the raw version of the dataset
df = load_or_create_df(
    version="raw",
    excel_path="../data/raw/HCP Dataset for Case Study.xlsx",
    sheet_name="Sheet1"
)

# Preview the data
df.head()

KeyboardInterrupt: 

In [ ]:
# Load from pickle (much faster than Excel)
df = pd.read_pickle("../data/staging/hcp_df.pkl")
df.head()

## 3. Feature Engineering – Episode Duration & Charge Per Day

In [2]:
# Convert dates to datetime format
df["AdmissionDate"] = pd.to_datetime(df["AdmissionDate"])
df["DischargeDate"] = pd.to_datetime(df["DischargeDate"])

# Calculate episode duration in days
df["EpisodeDuration"] = (df["DischargeDate"] - df["AdmissionDate"]).dt.days + 1

# Calculate charge per day
df["ChargePerDay"] = df["ChargeAmount"] / df["EpisodeDuration"]

# Preview the new columns
df[["EpisodeID", "EpisodeDuration", "ChargeAmount", "ChargePerDay"]]

KeyError: 'DischargeDate'

# 4. Feature Engineering – Referral Source Flags

In [3]:
# Create binary flags for key referral sources
df["IsEmergencyReferral"] = df["ReferralSource"].str.contains("Emergency", case=False)
df["IsExternalGPReferral"] = df["ReferralSource"].str.contains("External GP", case=False)

# Preview the new flags
df[["EpisodeID", "ReferralSource", "IsEmergencyReferral", "IsExternalGPReferral"]]

NameError: name 'df' is not defined